# Ejercicio LLMs 2: LangChain
## Aprende Machine Learning


Instalamos librerias que utilizaremos en la Notebook

In [ ]:
# Crea un environment por ej con conda
# !conda create -n ejercicio02 python=3.11
# Activa el environment
# !conda activate ejercicio02

# Instalar las dependencias
# !conda install ipykernel
# !conda install langchain==0.3.20 -c conda-forge
# !pip install langchain-openai langchain-community

# Para el agente
# !pip install -U wikipedia langchain_experimental numexpr duckduckgo-search ddgs


In [ ]:
from langchain.chat_models import ChatOpenAI

import os

if not os.environ.get("OPENAI_API_KEY"):
    # Usamos LM Studio en local
    llm = ChatOpenAI(
        openai_api_base="http://localhost:1234/v1",
        openai_api_key="lmstudio",
        )
else:
    # Usamos OpenAI en la nube
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        # api_key="...",
)

In [3]:
from IPython.display import Markdown

ai_msg = llm.invoke("Dime 5 sitios bonitos para visitar en Galicia")
md_text = ai_msg.content
Markdown(md_text)

¡Por supuesto! Aquí tienes cinco lugares hermosos que puedes visitar en Galicia, cada uno con su encanto único:

1. **Santiago de Compostela**: La capital espiritual del peregrinaje de Santiago es una ciudad rica en historia y patrimonio cultural. Su catedral, declarada Patrimonio de la Humanidad por la UNESCO, es un punto destacado y merece ser visitada.

2. **Pontevedra**: Esta ciudad tiene un encanto urbano que combina arquitectura histórica con modernas zonas verdes, como el parque de A Mañá. También cuenta con una playa, Praia da Ria, que es ideal para disfrutar del mar durante un paseo o una caminata.

3. **Finisterre**: Conocida como "el fin del mundo" por su promontorio de Finisterrae, es un lugar mágico y lleno de leyendas. La playa de Cabo Ortegal y la playa de Sanxenxo son destinos icónicos para observar el océano Atlántico.

4. **O Grove**: Este pequeño pueblo minero en las montañas de La Guardia es un lugar perfecto para explorar su rica historia minera, visitar las ruinas del antiguo yacimiento y disfrutar de paisajes impresionantes con vistas al mar.

5. **Coveiro**: En las Montañas do Candado, este pequeño pueblo ofrece una experiencia rural y tranquila en medio de la naturaleza. Ideal para quienes buscan un lugar menos turístico, es perfecto para pasear por sus caminos rurales o simplemente relajarse.

Espero que estos destinos te inspiren a planificar tu visita a Galicia pronto!

# Uso de Templates

In [29]:
from langchain.prompts import ChatPromptTemplate

template_string = "Dime {cantidad} sitios bonitos para visitar en {lugar}"

prompt_template = ChatPromptTemplate.from_template(template_string)

print(prompt_template.messages[0].prompt)

input_variables=['cantidad', 'lugar'] input_types={} partial_variables={} template='Dime {cantidad} sitios bonitos para visitar en {lugar}'


In [30]:
cantidad = "2"
lugar = "Roma"
message = prompt_template.format_messages(
                    cantidad=cantidad,
                    lugar=lugar)
print(message[0])

content='Dime 2 sitios bonitos para visitar en Roma' additional_kwargs={} response_metadata={}


In [32]:
# Ejecutamos el LLM
ai_msg = llm.invoke(message)
md_text = ai_msg.content
Markdown(md_text)

¡Claro! Aquí tienes dos lugares maravillosos que no puedes perderte cuando visites Roma:

1. **El Coliseo (Colosseo)**  
   - *Qué es*: El emblemático anfiteatro romano, símbolo de la ingeniería y la cultura del Imperio.  
   - *Por qué vale la pena visitarlo*: Puedes recorrer sus niveles, imaginar los combates de gladiadores y disfrutar de una vista panorámica de las ruinas circundantes. Además, el monumento ofrece visitas guiadas con audio en varios idiomas que cuentan la historia fascinante detrás de cada columna.

2. **La Plaza de San Pedro (Piazza San Pietro) y la Basílica de San Pedro**  
   - *Qué es*: El corazón espiritual del catolicismo, donde se reúne el Papa y millones de fieles. La plaza está diseñada por Gian Lorenzo Bernini, con columnas doradas que guían a los visitantes hacia la basílica.  
   - *Por qué vale la pena visitarlo*: Puedes admirar la arquitectura renombrada de la Basílica (con su enorme cúpula de Miguel Ángel), subir a la terraza para una vista impresionante del Vaticano y explorar el Tesoro de San Pedro, que alberga reliquias históricas y obras de arte sacro.

Ambos sitios combinan historia, arquitectura y un ambiente que te transporta directamente al corazón de Roma. ¡Disfruta tu viaje!

# Cadenas (chaining)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

txt_parser = StrOutputParser()

# Creamos la cadena
chain = prompt_template | llm | txt_parser

txt_msg = chain.invoke({"cantidad": "3", "lugar": "Mexico DF"})

Markdown(txt_msg)

Ciudad de México (antes conocida como México DF) tiene muchos lugares hermosos y llenos de historia que vale la pena visitar. Aquí te menciono tres sugerencias:

1. **Zócalo**: Es el corazón histórico de la ciudad, rodeado por importantes edificios como la Catedral Metropolitana, el Palacio Nacional y el Museo Nacional de Arte. El Zócalo es un excelente lugar para observar la arquitectura colonial y disfrutar del ambiente durante eventos culturales o festivos.

2. **Palacio de Bellas Artes**: Este edificio es una joya artística que combina elementos de varias estilos, como el neoclásico y el barroco mexicano. Dentro del palacio se encuentra un famoso museo de arte que tiene colecciones de pinturas de importantes artistas mexicanos.

3. **Plaza Garibaldi**: Conocida por su ambiente animado y música en vivo, especialmente mariachi, es un lugar ideal para degustar platillos tradicionales como tacos y mole y disfrutar la fiesta. La plaza también ofrece espectáculos de baile y shows culturales.

Estos son solo algunos de los muchos lugares que puedes visitar en Ciudad de México; cada uno tiene su propia historia y encanto único.

# Json Parser

In [8]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

prompt_template2 = PromptTemplate(
    template="Dime {cantidad} sitios bonitos para visitar en {lugar}.\n{format_instructions}",
    input_variables=["cantidad", "lugar"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)
chain = prompt_template2 | llm | json_parser

json_msg = chain.invoke({"cantidad": "3", "lugar": "Santiago de Chile"})
json_msg

{'sitios_bonitos': [{'nombre': 'Parque Forestal',
   'descripcion': 'Este parque es un oasis en medio de la ciudad, con hermosos caminos para pasear y una cascada que ofrece un bello panorama.'},
  {'nombre': 'Plaza de Armas',
   'descripcion': 'El corazón histórico de Santiago, esta plaza está rodeada por edificios coloniales e históricos, como el Palacio de La Moneda y la Catedral Metropolitana.'},
  {'nombre': 'Costanera Norte',
   'descripcion': 'Es una gran opción para disfrutar del aire fresco y admirar las vistas panorámicas de Santiago. También cuenta con áreas verdes y actividades al aire libre.'}]}

# Salida Estructurada: Objetos Pydantic

OJO! esto funciona sólo en algunos modelos, no es un estándard, por lo que puede fallar!.

Probado con el modelo (pago) gpt4o-mini funciona correctamente.

En local (gratis) funciona por ej. con el modelo gpt-oss-20b Instruct y DeepSeek

In [27]:
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

# Pydantic
class Punto_de_Interes(BaseModel):
    """Descripcion de cada punto de interés."""
    nombre: str = Field(description="Nombre del punto de interés.")
    descripcion: str = Field(description="Breve descripción del punto de interés y su relevancia.")

class Sitio(BaseModel):
    """Sitio turístico para visitar."""

    nombre: str = Field(description="Nombre del lugar.")
    puntos_de_interes: List[Punto_de_Interes] = Field(description="Lista que contiene la cantidad de puntos solicitados por el usuario.")

pydantic_parser = PydanticOutputParser(pydantic_object=Sitio)

prompt_template3 = PromptTemplate(
    template="Dime {cantidad} sitios bonitos para visitar en {lugar}.\nDevuelve un objeto JSON que siga exactamente este formato: {format_instructions}",
    input_variables=["cantidad", "lugar"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
)
chain = prompt_template3 | llm | pydantic_parser

objeto_sitio = chain.invoke({"cantidad":2, "lugar": "Miami"})
objeto_sitio

Sitio(nombre='Miami', puntos_de_interes=[Punto_de_Interes(nombre='South Beach', descripcion='La playa más conocida de Miami, conocida por sus arenas blancas y la línea de edificios Art Deco que rodean el agua. Un lugar perfecto para tomar fotos y disfrutar de la playa.'), Punto_de_Interes(nombre='Parque Nacional de los Everglades', descripcion='Un paisaje único donde se unen la ciudad y la naturaleza, conocido por su flora y fauna autóctona, como los aligátors. Es un destino ideal para el turismo ecológico.')])

In [28]:
objeto_sitio.puntos_de_interes[0].nombre

'South Beach'

# Memoria

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from collections import deque

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="Eres un asistente que responde preguntas en Español."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

# almacenará los últimos 50 mensajes
lista_de_mensajes = deque(maxlen=50)

lista_de_mensajes.append(HumanMessage(content="Hola, Mi color favorito es el azul."))

lista_de_mensajes.append(AIMessage(content="Hola ¿En qué puedo ayudarte?."))

lista_de_mensajes.append(HumanMessage(content="De que tamaño es una pelotita de tenis?"))

chain = prompt | llm

ai_msg = chain.invoke(
    {
        "messages": list(lista_de_mensajes)
    }
)
print(ai_msg.content)

Una pelota de tenis tiene un tamaño específico según las normas del deporte. El diámetro de la pelota debe estar comprendido entre **6,35 cm** y **6,39 cm**, lo cual equivale a aproximadamente **2,51 pulgadas**.

Si tienes alguna otra pregunta, ¡no dudes en decírmela!


In [12]:
# agrego a la memoria el mensaje previo
lista_de_mensajes.append(AIMessage(content=ai_msg.content))

# ponemos a prueba la memoria
lista_de_mensajes.append(HumanMessage(content="¿Cuál es mi color favorito?"))

ai_msg = chain.invoke(
    {
        "messages": list(lista_de_mensajes)
    }
)
print(ai_msg.content)

Tu color favorito es el azul, según mencionaste al principio. 😊 Si necesitas más información o ayuda sobre algo relacionado con el azul o cualquier otro tema, ¡estoy aquí para ayudarte!


# Flujos de Datos

# Secuencial

In [13]:
prompt_1 = ChatPromptTemplate.from_template(template="Crea tres títulos cortos y atrapantes para animar a quien lea a visitar {lugar}")

prompt_2 = ChatPromptTemplate.from_template(template="Crea un parrafo corto en base a estos títulos: {titulos}. Que el texto incluya puntos de interés. Al principio del parrafo; incluye el mejor título y descarta el resto.")

chain = prompt_1 | llm | {'titulos' : txt_parser} | prompt_2 | llm | txt_parser

articulo = chain.invoke({"lugar": "Sidney, Australia"})

Markdown(articulo)

**Sidney: Un Salto al Altamar de la Aventura**

Si estás buscando una escapada que te inunde de emociones y experiencias únicas, Sidney es tu próxima parada perfecta. Este paraíso urbano ofrece un sinfín de actividades que prometen adrenalina y diversión. Explora las playas de Manly, donde los surfistas expertos deslizan suavemente sobre olas majestuosas. No te quedes quieto en las calles animadas de Darling Harbour, llenas de mercados nocturnos y restaurantes exquisitos. Y si prefieres algo más tranquilo, la isla de Cockatoo es ideal para pasear por sus verdes caminos y relajarte bajo el sol. Sidney no solo te sorprende con su hermoso paisaje sino que también te invita a vivir una aventura inolvidable en cada paso que das.

## Paralelo

In [14]:
from langchain_core.runnables import RunnableLambda

prompt_1 = ChatPromptTemplate.from_template(template="Dame la población de {lugar}. Responde únicamente el número. Sin Markdown.")

prompt_2 = ChatPromptTemplate.from_template(template="Dime la comida típica de: {lugar}. Responde con muy pocas palabras. Sin Markdown.")

def combinar_salidas(inputs):
    return f"""Población: {inputs['salida_1']}\n
                Comida típica: {inputs['salida_2']}"""

chain_1 = prompt_1 | llm | txt_parser
chain_2 = prompt_2 | llm | txt_parser

chain = {'salida_1' : chain_1,
         'salida_2' : chain_2
         } | RunnableLambda(combinar_salidas)

salida = chain.invoke({"lugar": "Bélgica"})

print(salida)

Población: 11572000

                Comida típica: Waffles, fries con mayonesa, chocolate.


## Router / Bifurcación de flujo

In [15]:
prompt_1 = ChatPromptTemplate.from_template(template="Eres un experto sobre el Espacio y los planetas. Responde detalladamente a la siguiente pregunta: {pregunta}")

prompt_2 = ChatPromptTemplate.from_template(template="Eres muy fan del fútbol. Responde la pregunta dando analogías del mundo del fútbol: {pregunta}")

prompt_0 = ChatPromptTemplate.from_template(template="Dime si la siguiente pregunta es acerca de un planeta, astronomía o el espacio. Pregunta: {pregunta}. Responde únicamente con SI o NO.")

chain_1 = prompt_1 | llm | txt_parser
chain_2 = prompt_2 | llm | txt_parser
chain_0 = prompt_0 | llm | txt_parser

def router(input):
    es_planeta = chain_0.invoke({'pregunta': input["pregunta"]})

    if "SI" in es_planeta:
        print("Pregunta sobre planetas")
        return chain_1
    else:
        print("No es pregunta sobre planetas")
        return chain_2

router_chain = RunnableLambda(router)

salida = router_chain.invoke({"pregunta": "¿Cuántas lunas tiene Júpiter?"})

Markdown(salida)

Pregunta sobre planetas


Júpiter es el planeta con más lunas confirmadas en nuestro sistema solar, y actualmente se sabe que posee **149 lunas** registradas (aunque este número puede variar ligeramente ya que algunas lunas menores no siempre son reconocidas de forma oficial).

Sin embargo, para responder de manera detallada:

### Lunas principales:
Júpiter tiene cuatro grandes lunas descubiertas por Galileo en 1610, las llamadas **cuatro lunas gigantes** o **Galileanas**, que incluyen:
1. **Europa**: Es conocida por su gruesa capa de hielo y posibles océanos subsurfacos bajo ese hielo.
2. **Ganimedes**: La mayor luna del sistema solar, con una superficie diversa que incluye lagos de metano, geysers y vastas extensiones de hielo.
3. **Io**: Conocida por sus numerosos volcanes activos y su atmósfera de origen volcánico.
4. **Callisto**: La cuarta luna gigante, con un relieve marcado por crateras grandes y fajas de coloración variedad.

### Lunas medias:
Estas lunas tienen diámetros entre 10 km y unos pocos cientos de kilómetros, y comprenden a lunas como:
- Amaltea
- Adrastea
- Sinope
- Metis

### Lunas menores o pequeñas:
Estas son las lunas más pequeñas, con diámetros inferiores a 10 km. El sistema de Júpiter tiene una gran cantidad de estas lunas, que han sido descubiertas en campañas recientes.

Además, hay algunas categorías intermedias o subcategorizadas, como las lunas sin nombre (que aún no tienen un nombre oficial), y los satélites adquiridos por Júpiter tras su formación original. También es posible que nuevas lunas menores sean descubiertas en el futuro.

En resumen, aunque la cifra de 149 se considera actualmente correcta, siempre hay posibilidades de nuevos hallazgos o reevaluaciones del estado de algunas lunas.

# Agente sencillo

In [ ]:
from langchain.agents import load_tools, initialize_agent, Tool
from langchain.agents import AgentType, tool
from langchain.utilities import DuckDuckGoSearchAPIWrapper

In [17]:
duck = DuckDuckGoSearchAPIWrapper(region="es-es", max_results=5)

tools = load_tools(["llm-math","wikipedia"], llm=llm)

ducktool = [Tool(
        name="duckduckgo",
        func=duck.run,
        description="Util para realizar búsquedas en internet.",
    ),]


In [18]:
agent = initialize_agent(
    tools=tools + ducktool,
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    #max_iterations=10,
    verbose = True)

/var/folders/98/vy0n29r57p5ctr261ywmdg2r0000gn/T/ipykernel_39636/2459508955.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [20]:
pregunta = "Mafalda es una historieta famosa de la Argentina. ¿Cuando murió su autor, Quino? Usa las herramientas a tu disposición para responder."
result = agent(pregunta)



> Entering new AgentExecutor chain...
Thought: I need to find out the year Quino (Hernán Funes) died to answer the question.
Action:
```
{
  "action": "duckduckgo",
  "action_input": "Quino date of death"
}
```
Observation: Mafalda as a site of refuge : Quino 's comic, state repression, and audiences in ... Latinos mourn death of Quino , creator of "Mafalda" cartoon" . ... Quino and Brascó offered Mafalda ... Final years and death In 1990, Quino settled down in Spain and naturalized himself to become a Spanish citizen. The bulk of his career to date has been literary translation but in the 1980s and 1990s he was a comics editor at London-based British publisher ... date of death ... https://books.google.com.ar/books?id=t2-_DwAAQBAJ&pg=PT124&lpg=PT124&dq=max+moritz+prize+ quino &source=bl&ots ... The Adventures of Amina Al-Sirafi " is an entertaining read for anyone seeking a captivating blend of history, magic, and quirky badass ...
Thought:The information I found mentions Quino's se

In [21]:
result["output"]

'Quino died on September 30, 2020.'

## Define una herramienta propia

In [ ]:

@tool
def creador_de_motes(nombre: str) -> str:
    """Esta funcion recibe un nombre y devuelve un mote divertido."""
    return "Crack"

agent= initialize_agent(
    [creador_de_motes], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

result = agent("Cual es un buen mote para el nombre Alejandro?.")



> Entering new AgentExecutor chain...
Thought: Necesito usar la función creador_de_motes para generar un mote divertido.
Action:
```
{
  "action": "creador_de_motes",
  "action_input": "Alejandro"
}
```
Observation: Crack
Thought:I now know the final answer
Final Answer: Un buen mote para el nombre Alejandro podría ser "Crack".

> Finished chain.


In [23]:
result["output"]

'Un buen mote para el nombre Alejandro podría ser "Crack".'